<a href="https://colab.research.google.com/github/urmilapol/urmilapolprojects/blob/master/rssthyroid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! gdown --id 12zjXLKNkxdvdARvDLvXez8lJFb52lGJ5

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=12zjXLKNkxdvdARvDLvXez8lJFb52lGJ5
From (redirected): https://drive.google.com/uc?id=12zjXLKNkxdvdARvDLvXez8lJFb52lGJ5&confirm=t&uuid=a442c98b-bbb8-4b5b-a6ee-2da67d437154
To: /content/dataset.hdf5
100% 36.4G/36.4G [07:08<00:00, 84.9MB/s]


In [9]:
import h5py
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, TimeDistributed, Conv2D, MaxPooling2D, LSTM, concatenate
from tensorflow.keras.optimizers import Adam
import os

# --- 1. Data Loading ---

# Assuming 'dataset.hdf5' and 'metadata.csv' are in the same directory
HDF5_FILE = 'dataset.hdf5'
METADATA_FILE = 'metadata.csv'

def load_data(hdf5_path, metadata_path):
    """
    Loads cine-clip image data and metadata.
    Note: Loading the entire HDF5 can be memory intensive.
          Consider loading data in batches or on-the-fly during training.
    """
    try:
        with h5py.File(hdf5_path, 'r') as f:
            # Assuming 'cine_clips' is the key for the image data
            # The structure might be different, you'll need to inspect the HDF5 file
            # Example: images = f['cine_clips'][:]
            # For large datasets, you might iterate or use a custom data generator
            print(f"Keys in HDF5 file: {list(f.keys())}")
            # For demonstration, we'll assume a structure where 'images' and 'labels' are directly accessible
            # You will likely need to map metadata to the image data
            images = f['images'][:] # This is a placeholder, adapt based on actual HDF5 structure
            # Check the shape of your images. It should be (num_clips, num_frames, height, width, channels)
            print(f"Shape of loaded images: {images.shape}")

        metadata_df = pd.read_csv(metadata_path)
        print(f"Shape of metadata: {metadata_df.shape}")
        return images, metadata_df
    except Exception as e:
        print(f"Error loading data: {e}")
        print("Please ensure 'dataset.hdf5' and 'metadata.csv' are in the correct path "
              "and inspect the HDF5 file structure to access the correct keys.")
        return None, None

# Example usage:
# images, metadata_df = load_data(HDF5_FILE, METADATA_FILE)

# For the purpose of a runnable example, let's create dummy data
# In a real scenario, you'd load the actual data
num_clips = 100
frames_per_clip = 30
img_height = 128
img_width = 128
channels = 1 # Grayscale ultrasound images

images = np.random.rand(num_clips, frames_per_clip, img_height, img_width, channels).astype(np.float32)
metadata_df = pd.DataFrame({
    'patient_id': [f'P{i}' for i in range(num_clips)],
    'nodule_id': [f'N{i}' for i in range(num_clips)],
    'age': np.random.randint(20, 80, num_clips),
    'gender': np.random.choice(['M', 'F'], num_clips),
    'lesion_size_mm': np.random.uniform(5, 50, num_clips),
    'ti_rads': np.random.randint(2, 6, num_clips), # TI-RADS 2 to 5
    'histopathological_diagnosis': np.random.choice(['benign', 'malignant'], num_clips, p=[0.8, 0.2])
})
print("Using dummy data for demonstration.")
print(f"Dummy images shape: {images.shape}")
print(f"Dummy metadata shape: {metadata_df.shape}")


# --- 2. Preprocessing ---

def preprocess_metadata(df):
    """
    Encodes categorical features and scales numerical features.
    """
    # Encode 'gender'
    le_gender = LabelEncoder()
    df['gender_encoded'] = le_gender.fit_transform(df['gender'])

    # Scale numerical features (age, lesion_size_mm, ti_rads)
    scaler = StandardScaler()
    df[['age_scaled', 'lesion_size_mm_scaled', 'ti_rads_scaled']] = \
        scaler.fit_transform(df[['age', 'lesion_size_mm', 'ti_rads']])

    # Encode target variable
    le_diagnosis = LabelEncoder()
    df['diagnosis_encoded'] = le_diagnosis.fit_transform(df['histopathological_diagnosis'])
    # Malignant will likely be 1, benign 0 (check mapping with le_diagnosis.classes_)
    print(f"Diagnosis classes: {le_diagnosis.classes_}")

    return df, le_diagnosis

metadata_df, label_encoder = preprocess_metadata(metadata_df.copy()) # Use a copy to avoid modifying original df

# Assuming you have a way to map images to their corresponding metadata entry
# For this dummy data, we assume a 1:1 mapping by index
image_data = images # This would be your actual loaded cine-clip data
clinical_data = metadata_df[['age_scaled', 'gender_encoded', 'lesion_size_mm_scaled', 'ti_rads_scaled']].values
labels = metadata_df['diagnosis_encoded'].values

# Split data into training and testing sets
X_img_train, X_img_test, X_clin_train, X_clin_test, y_train, y_test = train_test_split(
    image_data, clinical_data, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"X_img_train shape: {X_img_train.shape}")
print(f"X_clin_train shape: {X_clin_train.shape}")
print(f"y_train shape: {y_train.shape}")


# --- 3. Model Building (Deep Learning for Cine-clips + Clinical Data) ---

def build_hybrid_model(input_shape_images, input_shape_clinical):
    """
    Builds a hybrid deep learning model for disease prediction.
    Combines a 3D CNN (for cine-clips) with a dense layer for clinical features.
    """
    # Image Input Branch (3D CNN for cine-clips)
    img_input = Input(shape=input_shape_images, name='image_input')
    x = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(img_input)
    x = MaxPooling3D((2, 2, 2), padding='same')(x)
    x = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D((2, 2, 2), padding='same')(x)
    x = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D((2, 2, 2), padding='same')(x)
    x = Flatten()(x)
    img_features = Dense(128, activation='relu')(x)
    img_features = Dropout(0.5)(img_features)

    # Clinical Data Input Branch
    clinical_input = Input(shape=(input_shape_clinical,), name='clinical_input')
    y = Dense(32, activation='relu')(clinical_input)
    clinical_features = Dropout(0.2)(y)

    # Concatenate features from both branches
    combined_features = concatenate([img_features, clinical_features])

    # Output Layer
    z = Dense(64, activation='relu')(combined_features)
    z = Dropout(0.3)(z)
    output = Dense(1, activation='sigmoid', name='output_diagnosis')(z) # Sigmoid for binary classification

    model = Model(inputs=[img_input, clinical_input], outputs=output)
    return model

# Define input shapes
input_shape_images = (frames_per_clip, img_height, img_width, channels)
input_shape_clinical = clinical_data.shape[1] # Number of clinical features

model = build_hybrid_model(input_shape_images, input_shape_clinical)
model.summary()

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# --- 4. Training ---

# You might need to adjust batch_size and epochs based on your hardware and dataset size
batch_size = 8 # Smaller batch size for cine-clips due to memory
epochs = 10

# Create TensorFlow datasets for better performance and memory management
train_dataset = tf.data.Dataset.from_tensor_slices(
    ({"image_input": X_img_train, "clinical_input": X_clin_train}, y_train)
).shuffle(buffer_size=1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices(
    ({"image_input": X_img_test, "clinical_input": X_clin_test}, y_test)
).batch(batch_size).prefetch(tf.data.AUTOTUNE)


print("\nStarting model training...")
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=test_dataset
)
print("Model training complete.")


# --- 5. Evaluation ---

print("\nEvaluating model performance on test set...")
loss, accuracy, auc = model.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")



Using dummy data for demonstration.
Dummy images shape: (100, 30, 128, 128, 1)
Dummy metadata shape: (100, 7)
Diagnosis classes: ['benign' 'malignant']
X_img_train shape: (80, 30, 128, 128, 1)
X_clin_train shape: (80, 4)
y_train shape: (80,)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 30, 128,   │          0 │ -                 │
│ (InputLayer)        │ 128, 1)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d (Conv3D)     │ (None, 30, 128,   │        896 │ image_input[0][0] │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d       │ (None, 15, 64,    │          0 │ conv3d[0][0]      │
│ (MaxPooling3D)      │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_1 (Conv3D)   │ (None, 15, 64,    │     55,360 │ max_pooling3d[0]… │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_1     │ (None, 8, 32, 32, │          0 │ conv3d_1[0][0]    │
│ (MaxPooling3D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 8, 32, 32, │    221,312 │ max_pooling3d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_2     │ (None, 4, 16, 16, │          0 │ conv3d_2[0][0]    │
│ (MaxPooling3D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 131072)    │          0 │ max_pooling3d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ clinical_input      │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │ 16,777,344 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │        160 │ clinical_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 32)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 160)       │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │     10,304 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_diagnosis    │ (None, 1)         │         65 │ dropout_2[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,065,441 (65.10 MB)

 Trainable params: 17,065,441 (65.10 MB)

 Non-trainable params: 0 (0.00 B)


Starting model training...
Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 261s 25s/step - accuracy: 0.6832 - auc: 0.3981 - loss: 0.6639 - val_accuracy: 0.8000 - val_auc: 0.4453 - val_loss: 0.5270
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 262s 26s/step - accuracy: 0.8072 - auc: 0.8004 - loss: 0.4757 - val_accuracy: 0.8000 - val_auc: 0.4297 - val_loss: 0.5201
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 260s 26s/step - accuracy: 0.8357 - auc: 0.5023 - loss: 0.4951 - val_accuracy: 0.8000 - val_auc: 0.4844 - val_loss: 0.5035
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 253s 25s/step - accuracy: 0.8464 - auc: 0.5030 - loss: 0.4594 - val_accuracy: 0.8000 - val_auc: 0.5312 - val_loss: 0.5017
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 259s 26s/step - accuracy: 0.7939 - auc: 0.3739 - loss: 0.5656 - val_accuracy: 0.8000 - val_auc: 0.5000 - val_loss: 0.5129
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 253s 25s/step - accuracy: 0.8239 - auc: 0.5457 - loss: 0.4886 - val_accuracy: 0.8000 - val_auc: 0.4844 - val_loss: 0.5027
Epoch 7/10
1

In [10]:
# --- 6. Prediction Example ---

def predict_new_patient(model, new_image_clip, new_clinical_data, label_encoder):
    """
    Makes a prediction for a single new patient.
    """
    # Ensure inputs are in the correct batch format for the model
    new_image_clip_batch = np.expand_dims(new_image_clip, axis=0)
    new_clinical_data_batch = np.expand_dims(new_clinical_data, axis=0)

    prediction_proba = model.predict([new_image_clip_batch, new_clinical_data_batch])[0][0]
    predicted_class_idx = (prediction_proba > 0.5).astype(int)
    predicted_diagnosis = label_encoder.inverse_transform([predicted_class_idx])[0]

    return predicted_diagnosis, prediction_proba

# Example new data (replace with actual new patient data)
# Remember to preprocess new clinical data using the same scalers/encoders
# used during training!
sample_index = np.random.randint(0, len(X_img_test))
sample_image_clip = X_img_test[sample_index]
sample_clinical_data = X_clin_test[sample_index]
true_diagnosis = label_encoder.inverse_transform([y_test[sample_index]])[0]

predicted_diagnosis, prediction_proba = predict_new_patient(
    model, sample_image_clip, sample_clinical_data, label_encoder
)

print(f"\n--- Prediction for a Sample Patient ---")
print(f"True Diagnosis: {true_diagnosis}")
print(f"Predicted Diagnosis: {predicted_diagnosis}")
print(f"Prediction Probability (Malignant): {prediction_proba:.4f}")

# You can also save your trained model
# model.save('thyroid_nodule_detection_model.h5')
# print("Model saved to thyroid_nodule_detection_model.h5")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 714ms/step

--- Prediction for a Sample Patient ---
True Diagnosis: benign
Predicted Diagnosis: benign
Prediction Probability (Malignant): 0.2438
